In [68]:
import subprocess

In [69]:
from recipe_100ppt import spent_232, spent_no232, ff_232, spentff_232, ff_no232, spentff_no232

In [70]:
# Separations
sourc1 = {'name' : 'Source1',
          'config' : {'Source' : {'outcommod' : 'SpentFuel1',
                                  'outrecipe' : 'SpentUOX_232',
                                  'throughput' : 1e10}}
         }
sourc2 = {'name' : 'Source2',
          'config' : {'Source' : {'outcommod' : 'SpentFuel2',
                                  'outrecipe' : 'SpentUOX_no232',
                                  'throughput' : 1e10}}
         }
streams = {'item' : [{'commod' : 'Pu',
                      'info' : {'buf_size' : 1e100,
                                'efficiencies' : {'item' : {'comp': 'Pu', 'eff': 0.99}}}},
                     {'commod' : 'RU', 
                      'info' : {'buf_size' : 1e100, 
                                'efficiencies' : {'item' : {'comp': 'U', 'eff': 0.99}}}}
                    ]
          }
sep = {'name' : 'Sep',
       'config' : {'Separations' : {'leftover_commod' : 'Waste',
                                    'throughput' : 1e100,
                                    'feedbuf_size' : 1e100,
                                    'feed_commods' : {'val' : ['SpentFuel1', 'SpentFuel2', 
                                                               'StoredFF_Add', 'StoredFF_Non']},
                                    'feed_commod_prefs' : {'val' : [1, 1, 2, 2]},
                                    'streams' : streams}}
      }
sink = {'name' : 'Waste',
        'config' : {'Sink' : {'in_commods' : {'val' : 'Waste'}}}
       }

In [71]:
# Mixer

pu = {'info' : {'mixing_ratio' : 0.01,
                'buf_size' : 1e8}, 
                'commodities' : {'item' : {'commodity' : 'Pu', 'pref' : 1}}
     }
ru = {'info' : {'mixing_ratio' : 0.99,
                'buf_size' : 1e9}, 
                'commodities' : {'item' : {'commodity' : 'RU', 'pref' : 1}}
     }
mx1 = {'name' : 'Mix1',
       'config' : {'Mixer' : {'in_streams' : {'stream' : [pu, ru]},
                              'out_commod' : 'FF_Add',
                              'throughput' : 1e10,
                              'out_buf_size' : 1e20}}
      }
mx2 = {'name' : 'Mix2',
       'config' : {'Mixer' : {'in_streams' : {'stream' : [pu, ru]},
                              'out_commod' : 'FF_Non',
                              'throughput' : 1e10,
                              'out_buf_size' : 1e20}}
      }

In [72]:
# Post-Mixer

rxt = {'name' : 'Rxtr', 
       'lifetime' : 960,
       'config' : {'Reactor' : {'fuel_incommods' :  {'val' : ['FF_Add', 'FF_Non']},
                                'fuel_outcommods' : {'val' : ['SpentFF_Add', 'SpentFF_Non']},
                                'fuel_inrecipes' :  {'val' : ['FF_232', 'FF_no232']},
                                'fuel_outrecipes' : {'val' : ['SpentFF_232', 'SpentFF_no232']},
                                'fuel_prefs' : {'val' : [1, 2]},
                                'cycle_time' : 3,
                                'refuel_time' : 0,
                                'assem_size' : 7490,
                                'n_assem_core' : 5,
                                'n_assem_batch' : 1,
                                'power_cap' : 400
                               }
                  }
      }

stor1 = {'name' : 'Storage1',
         'config' : {'Storage' : {'in_commods' : {'val' : 'SpentFF_Add'},
                                  'out_commods' : {'val' : 'StoredFF_Add'},
                                  'in_recipe' : 'SpentFF_232',
                                  'residence_time' : 1
                                 }
                    }
        }
stor2 = {'name' : 'Storage2',
         'config' : {'Storage' : {'in_commods' : {'val' : 'SpentFF_Non'},
                                  'out_commods' : {'val' : 'StoredFF_Non'},
                                  'in_recipe' : 'SpentFF_no232',
                                  'residence_time' : 1
                                 }
                    }
        }

In [73]:
control = {'duration' : 20, 
           'startmonth' : 1, 
           'startyear' : 2022
          }
archetypes = {'spec' : [{'lib' : 'cycamore', 'name' : 'Source'},
                        {'lib' : 'cycamore', 'name' : 'Separations'},
                        {'lib' : 'cycamore', 'name' : 'Mixer'},
                        {'lib' : 'cycamore', 'name' : 'Storage'},
                        {'lib' : 'cycamore', 'name' : 'Reactor'},
                        {'lib' : 'cycamore', 'name' : 'Sink'},
                        {'lib' : 'cycamore', 'name' : 'ManagerInst'},
                        {'lib' : 'cycamore', 'name' : 'GrowthRegion'},
                       ]
             }
lin_func = {'piece' : [{'start' : 2,
                        'function' : {'type' : 'linear',
                                      'params' : '0 800'
                                     }
                       }]
           }

growth_region = {'GrowthRegion' : {'growth' : {'item' : [{'commod' : 'power',
                                                          'piecewise_function' : lin_func
                                                         }]
                                              }
                                  }
                }
region = {'name' : 'Region', 
          'config' : growth_region, 
          'institution' : {'name' : 'Inst',
                           'initialfacilitylist' : {'entry' : [{'number' : 1, 'prototype' : 'Source1'},
                                                               {'number' : 1, 'prototype' : 'Source2'},
                                                               {'number' : 1, 'prototype' : 'Sep'},
                                                               {'number' : 1, 'prototype' : 'Mix1'},
                                                               {'number' : 1, 'prototype' : 'Mix2'},
                                                               {'number' : 1, 'prototype' : 'Rxtr'},
                                                               {'number' : 1, 'prototype' : 'Storage1'},
                                                               {'number' : 1, 'prototype' : 'Storage2'},
                                                               {'number' : 1, 'prototype' : 'Waste'}
                                                              ]},
                           'config' : {'ManagerInst' : {'prototypes' : {'val' : 'Rxtr'}}}
                          }
         }
sim = {'simulation' : {'control' : control,
                       'archetypes' : archetypes,
                       'region' : region,
                       'facility' : [sourc1, sourc2, sep, mx1, mx2, rxt, stor1, stor2, sink],
                       'recipe' : [spent_232, spent_no232, ff_232, spentff_232, ff_no232, spentff_no232]
                      }
      }


In [74]:
filebase = 'sep_failure'

in_file = filebase + '.py'
sim_file = '../output/' + filebase + '.sqlite'

with open(in_file, 'w') as file: 
    file.write('SIMULATION = ' + str(sim))
subprocess.run(['rm', sim_file])
subprocess.run(['cyclus', in_file, '-o', sim_file])

CompletedProcess(args=['cyclus', 'sep_failure.py', '-o', '../output/sep_failure.sqlite'], returncode=0)